In [1]:
import sys
sys.path.insert(0, '..')

In [2]:
from datetime import datetime
from typing import Dict, List
import os
import pandas as pd
from pandas import DataFrame, Series
import wandb
import pycountry
from flood_forecast.time_model import PyTorchForecast
from flood_forecast.trainer import train_function
from flood_forecast.config_sample import make_config_file
from wandb.wandb_run import Run

In [3]:
df: DataFrame = pd.read_csv('../data/sorgenia_wind_copernicus.csv')
df.head()

,time,UP_MPNTLCDMRN_1,UP_MPNTLCSMBC_1,UP_PEPIZZA_1,UP_PRCLCDMZRD_1,UP_PRCLCDMZRD_2,UP_PRCLCDPLRM_1,UP_PRCLCDPRZZ_1,UP_PRCLCMINEO_1,dew_point_2m_C_UP_MPNTLCDMRN_1,...,Year cos,t,days_from_start,hour,day,day_of_week,month,hours_from_start,categorical_day_of_week,categorical_hour
0,2016-12-31 23:00:00+00:00,225.000,2070.0000,878.25,0.0,NaN,681.300,217.875,0.0,-0.902831,...,0.999953,0.0,0,23,31,5,12,0.0,5,23
1,2017-01-01 00:00:00+00:00,65.625,2606.4375,1096.50,0.0,NaN,797.325,159.225,0.0,-0.783022,...,0.999946,1.0,0,0,1,6,1,1.0,6,0
2,2017-01-01 01:00:00+00:00,31.875,2886.1875,1380.75,0.0,NaN,682.500,116.550,0.0,-0.565645,...,0.999939,2.0,0,1,1,6,1,2.0,6,1
3,2017-01-01 02:00:00+00:00,0.000,1576.5000,145.50,0.0,NaN,280.425,96.525,0.0,-0.331075,...,0.999930,3.0,0,2,1,6,1,3.0,6,2
4,2017-01-01 03:00:00+00:00,0.000,2340.5625,155.25,0.0,NaN,126.450,76.575,0.0,-0.668040,...,0.999922,4.0,0,3,1,6,1,4.0,6,3


In [4]:
df.shape

(34729, 113)

In [8]:
len(df.columns[1:-2])

110

In [11]:
# index: Series = df['days_from_start']
# df_test = df.loc[index >= int(index.max()*0.9)]

In [ ]:
# Config file for WanDB sweeps

def make_config_file(file_path: str, df_len: int) -> Dict:
    train_number: float = df_len * .7
    validation_number: float = df_len *.9
    config_default={
      "model_name": "DecoderTransformer",
      "model_type": "PyTorch",
      "takes_target": False,
      "model_params": {
      "n_time_series":110,
      "n_head": 8,
      "forecast_history":168,
      "n_embd": 1, 
      "num_layer": 5,
      "dropout":0.1,
      "q_len": 1,
      "scale_att": False,
      "forecast_length": 12, 
      "additional_params":{}
     },
     "dataset_params":
     {
         "class": "default",
          "training_path": file_path,
          "validation_path": file_path,
          "test_path": file_path,
          "batch_size":64,
          "forecast_history":168,
          "forecast_length":12,
          "train_end": int(train_number),
          "valid_start":int(train_number+1),
          "valid_end": int(validation_number),
          "target_col": ['UP_MPNTLCDMRN_1', 'UP_MPNTLCSMBC_1', 'UP_PEPIZZA_1',
                       'UP_PRCLCDMZRD_1', 'UP_PRCLCDMZRD_2', 'UP_PRCLCDPLRM_1',
                       'UP_PRCLCDPRZZ_1', 'UP_PRCLCMINEO_1'],
          "relevant_cols": list(df.columns[1:-2]),
          "scaler": "StandardScaler", 
          "interpolate": True,
          "sort_column":"time",
     },
     "training_params":
      {
        "criterion":"DilateLoss",
        "optimizer": "Adam",
        "optim_params":
        {
        },
        "lr": 0.001,
        "epochs": 4,
        "batch_size":64
      },
      "early_stopping": {
          "patience":3
      },
      "GCS": False,
      "sweep":False,
      "wandb":False,
      "forward_params":{},
      "metrics":["DilateLoss"],
      "inference_params":
        {     
              "datetime_start":"2020-08-06 23:00:00",
                "hours_to_forecast": 1344, 
                "test_csv_path":file_path,
                "decoder_params":{
                    "decoder_function": "simple_decode", 
                  "unsqueeze_dim": 1
                },
                "dataset_params":{
                  "file_path": file_path,
                  "forecast_history":168,
                  "forecast_length":12,
                  "relevant_cols": list(df.columns[1:-2]),
                  "target_col": ['UP_MPNTLCDMRN_1', 'UP_MPNTLCSMBC_1', 'UP_PEPIZZA_1',
                               'UP_PRCLCDMZRD_1', 'UP_PRCLCDMZRD_2', 'UP_PRCLCDPLRM_1',
                               'UP_PRCLCDPRZZ_1', 'UP_PRCLCMINEO_1'],
                  "scaling": "StandardScaler",
                  "interpolate_param": True
                }
          },
    }

    return config_default